In [1]:
import os 
import librosa 
import pathlib
import wavio as wv
import numpy as np
import tensorflow as tf
from scipy.io.wavfile import read

In [136]:
# path to the dataset
data_dir = "C:/Project/FINAL-DATASET/yes"
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
filenames = tf.io.gfile.glob(str(data_dir) + '/*')

def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-1]

for i in range(0 , len(filenames)):
    samples = read(filenames[i])

In [144]:
def augment(samples):
    augmentation = samples.copy()
     # Noise Addition
    noise_amp = 0.075*np.random.uniform()*np.amax(augmentation)
    augmentation = augmentation.astype('float64') + noise_amp * np.random.normal(size=augmentation.shape[0])
    
     # Random Shifting
    s_range = int(np.random.uniform(low=-5, high = 5)*1000)
    augmentation = np.roll(samples, s_range)
    
     # Stretching The Sound
    augmentation = librosa.effects.time_stretch(samples, 0.8)
    
     # Pitch Tuning
    bins_per_octave = 12
    pitch_pm = 2
    sample_rate = 16000
    pitch_change =  pitch_pm * 2*(np.random.uniform())
    augmentation = librosa.effects.pitch_shift(samples.astype('float64'), 
                                       sample_rate, n_steps=pitch_change, 
                                       bins_per_octave=bins_per_octave)
    
     # Random Value Change
    dyn_change = np.random.uniform(low=-0.5 ,high=7)  # default low = 1.5, high = 3
    augmentation = samples*dyn_change
    
     # Speed and Pitch Tuning
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.0  / length_change
    tmp = np.interp(np.arange(0,len(augmentation),speed_fac),np.arange(0,len(augmentation)),augmentation)
    minlen = min(augmentation.shape[0], tmp.shape[0])
    augmentation *= 0
    augmentation[0:minlen] = tmp[0:minlen]
    
    return augmentation

In [145]:
for i in range(0 , len(filenames)):
    samples = read(filenames[i])
    samples = np.array(samples[1], dtype=float)
    samples = np.float32(samples)
    augmented_sample = augment(samples)
    freq = 16000
    x = get_label(filenames[i]).numpy().decode("utf-8")
    # file variable stores the string which is the path to the dataset and is added with a prefix (aug4_) to seperate all the augmented files from the normal files
    wv.write(file="C:/Users/HP/Desktop/My_Folder/DATASET-FINAL-IFM/Yes/" + "aug4_" + x , 
             data=augmented_sample, 
             rate=freq, 
             sampwidth=2)